<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/2020m0/2%20Advanced/024%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E5%85%A5%E5%87%BA%E5%8A%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 複雑なファイルの入出力

研究でPythonを使う状況を考えると、何はともあれ大きなデータをファイルから読みこんで、処理して、ファイルに出力するという使い方がほとんどになるだろう。あらかじめPythonで読みこむつもりで書かれたデータばかりではないので、読み込むのにも工夫が必要になる。また、Pythonの出力を他のソフトウェアで利用する場合には、そのソフトウェアにあわせたデータ出力が求められる。



## ファイル入力

Google Drive上にファイルがあるという前提で話をすすめる。

ファイルはリンク先からダウンロードし、Google Drive内の`fileio/`という名前のフォルダーに入れなさい。

* [data1.txt](https://raw.githubusercontent.com/vitroid/PythonTutorials/2020m0/2%20Advanced/fileio/data1.txt)
* [data2.txt](https://raw.githubusercontent.com/vitroid/PythonTutorials/2020m0/2%20Advanced/fileio/data2.txt)
* [data3.txt](https://raw.githubusercontent.com/vitroid/PythonTutorials/2020m0/2%20Advanced/fileio/data3.txt)
* [data4.txt](https://raw.githubusercontent.com/vitroid/PythonTutorials/2020m0/2%20Advanced/fileio/data4.txt)
* [data5.mdvw](https://raw.githubusercontent.com/vitroid/PythonTutorials/2020m0/2%20Advanced/fileio/data5.mdvw)

`fileio/data1.txt`の中身はこんな感じ。

```txt
1 2 3
4 5 6
7 8 9
10 11 12
```

In [ ]:
# Google DriveをColabに連携させる。

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#座標の羅列を読みこむ
file = open("/content/drive/My Drive/fileio/data1.txt")
coord = []
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    columns = line.split()  #空白やタブで行をカラムに分割する。
    print("columns=",columns)
    x, y, z = float(columns[0]),float(columns[1]),float(columns[2])
    print("x,y,z=",x,y,z)
    coord.append((x,y,z))
    print("coord:",coord)
print(coord)
print(coord[1][1])

### もうちょっとシンプルに

In [ ]:
#座標の羅列を読みこむ
file = open("/content/drive/My Drive/fileio/data1.txt")
coord = []
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    values = [float(x) for x in line.split()]  #分割して実数化する
    print(line,values)
    coord.append(values)
print(coord)


### 最初の3行は読みとばす

In [ ]:
file = open("/content/drive/My Drive/fileio/data2.txt")
for i in range(3):
    line = file.readline()
    print(line, end="")

coord = []
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    values = [float(x) for x in line.split()]  #分割して実数化する
    coord.append(values)
print(coord)

### 行のはじめに"@n"と書いてあり、そのあとのn行が座標データ

In [4]:
# file = open("/content/drive/My Drive/fileio/data3.txt")
file = open("fileio/data3.txt") #run Locally

# iter()は、第一引数の関数をつかってデータを取得し、その内容が第二引数と一致すると処理を終えるiterator。
for line in iter(file.readline, ""):
    if line[0] == "@":
        n = int(line[1:])
        coord = []
        for i in range(n):
            line = file.readline()
            values = [float(x) for x in line.split()]  #分割して実数化する
            coord.append(values)
        break               #whileループを抜ける
print(coord)

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0], [10.0, 11.0, 12.0]]


### \#以降はコメントとして無視する

In [5]:
# "#"よりもあとを除去する
def elim_comment(s):
    pos = s.find("#")
    if 0 <= pos:
        return s[0:pos]
    return s
    

# file = open("/content/drive/My Drive/fileio/data4.txt")
file = open("fileio/data4.txt")
for line in iter(file.readline, ""):
    if line[0] == "@":
        coord = []
        n = int(line[1:])
        for i in range(n):
            line = elim_comment(file.readline())
            values = [float(x) for x in line.split()]  #分割して実数化する
            coord.append(values)
        break               #whileループを抜ける

print(coord)

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0], [10.0, 11.0, 12.0]]


### 行ごとではなく、列ごとに読みこむ

In [6]:
#座標の羅列を読みこむ
# file = open("/content/drive/My Drive/fileio/data1.txt")
file = open("fileio/data1.txt")

# データを入れる先をあらかじめ3列分準備しておく。 
columns = [[] for i in range(3)]
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    values = [float(x) for x in line.split()]  #分割して実数化する
    for i in range(3):
        columns[i].append(values[i])
print(columns)

[[1.0, 4.0, 7.0, 10.0], [2.0, 5.0, 8.0, 11.0], [3.0, 6.0, 9.0, 12.0]]


### 行ごとに読みこんでから、転置する

In [7]:
s=[[1,2,3],[1,2,3,4],[1,2,3,4,5]]
m = max([len(x) for x in s])
m

5

In [ ]:
def transpose(matrix, default=None):
    #列数の最大値を調べる。
    maxc = max([len(x) for x in matrix])
    #コラムの数は列数の最大値
    columns = [[] for i in range(maxc)]
    for row in matrix:
        for i in range(maxc):
            if i < len(row):
                columns[i].append(row[i])
            else:
                #データが足りない部分はdefault値を入れる
                columns[i].append(default)
    return columns
    
file = open("/content/drive/My Drive/fileio/data1.txt")
coord = []
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    values = [float(x) for x in line.split()]  #分割して実数化する
    coord.append(values)
print(coord)
columns = transpose(coord)
print(columns)
data = ["list","of","strings"]
print(data)
print(transpose(data,default=""))

## ファイル出力

読みこみと同じように、Google Driveを使えば、ファイルが手許にあるのか、クラウドにあるのかを意識する必要はなくなる。

### テキストファイルの書き出し

write関数を使う。printと違い、改行しないので、改行文字`\n`を明示的に付ける必要がある。


In [ ]:
# "w"は書き込みのためにファイルを開くことを意味する。
file = open("/content/drive/My Drive/fileio/output.txt", "w")

file.write("Hello world!\n") # "\"はWindowsではYen markかも。
file.close()  # 開いたファイルは閉じる必要がある。閉じるまでファイルに書かれないことも。

書きだした内容は、しばらくすると自分のパソコンのGoogle Driveにも同期される。また、スマホのGoogle Driveアプリからもファイルができたことをチェックできる。

閉じ忘れを防ぐために、最近はwith文を使って書くことが推奨されている。

In [ ]:
with open("/content/drive/My Drive/fileio/output.txt", "w") as file:
    file.write("Hello world!\n") # "\"はWindowsではYen markかも。

# withはインデントブロックになるので、withのブロックが終わると自動的にファイルが閉じられる。

何行も書きだしたい場合はwithブロックの中でループを書く。


In [ ]:
with open("/content/drive/My Drive/fileio/output2.txt", "w") as file:
    for i in range(10):
        file.write(i, "\n") # これはエラーになるらしい。print文と違い、()の中にカンマで書きならべられない。


In [ ]:
with open("/content/drive/My Drive/fileio/output2.txt", "w") as file:
    for i in range(10):
        file.write("{0}\n".format(i)) # 一番上品な書き方。format構文であらかじめ整形した文字列を書きだす。


`format`文の使い方はまたいずれ。

## 宿題

data5.mdvwから、原子の座標と思われる部分だけを読みこんで、座標のリストを作って下さい。

それができたら、原子の名前も読みこみ、原子の種類ごとに、座標を別のリストに入れるようにしてみましょう。